## Generate Figure for Utah Meeting in Jan 9th 2023

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

import tTEM_toolbox as tt
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
import plotly.io as pio
pio.kaleido.scope.chromium_args = tuple([arg for arg in pio.kaleido.scope.chromium_args if arg != "--disable-dev-shm-usage"])

In [2]:
workdir = Path(r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test')
welllog = workdir.joinpath(r'Plot_with_well_log\Well_log.xlsx')
elevation = workdir.joinpath(r'well_Utah\usgs_water_elevation.csv')
ttemname_north = workdir.joinpath(r'Plot_with_well_log\PD1_I01_MOD.xyz')
ttemname_center = workdir.joinpath(r'Plot_with_well_log\PD22_I03_MOD.xyz')
ttem_lslake = workdir.joinpath(r'Plot_with_well_log\lsll_I05_MOD.xyz')
DOI = workdir.joinpath(r'Plot_with_well_log\DOID1_DOIStaE.xyz')
well_info = workdir.joinpath(r'well_Utah\USGSdownload\NWISMapperExport.xlsx')
colorRes = [[0, 'rgb(0,0,190)'],
            [1 / 16, 'rgb(0,75,220)'],
            [2 / 16, 'rgb(0,150,235)'],
            [3 / 16, 'rgb(0,200,255)'],
            [4 / 16, 'rgb(80,240,255)'],
            [5 / 16, 'rgb(30,210,0)'],
            [6 / 16, 'rgb(180,255,30)'],
            [7 / 16, 'rgb(255,255,0)'],
            [8 / 16, 'rgb(255,195,0)'],
            [9 / 16, 'rgb(255,115,0)'],
            [10 / 16, 'rgb(255,0,0)'],
            [11 / 16, 'rgb(255,0,120)'],
            [12 / 16, 'rgb(140,40,180)'],
            [13 / 16, 'rgb(165,70,220)'],
            [14 / 16, 'rgb(195,130,240)'],
            [15 / 16, 'rgb(230,155,240)'],
            [1, 'rgb(230,155,255)']]

In [3]:
def split_ttem(ttem_df, gwsurface_result):
    def get_distance(group1, group2):
        dis = np.sqrt((group1[0] - group2[0]) ** 2 + (group1[1] - group2[1]) ** 2)
        return dis
    abv_water_table = []
    blw_water_table = []
    ttem_groups = ttem_df.groupby(['UTMX', 'UTMY'])
    well_location = gwsurface_result[['UTMX', 'UTMY']].values
    for name, group in ttem_groups:
        ttem_xy = list(group[['UTMX', 'UTMY']].iloc[0])
        ttem_well_distance = list(map(lambda x: get_distance(ttem_xy, x), well_location))
        match = gwsurface_result.iloc[ttem_well_distance.index(min(ttem_well_distance))]
        elevation = match['water_elevation']
        ttem_abv = group[group['Elevation_End'] >= elevation]
        abv_water_table.append(ttem_abv)
        ttem_blw = group[group['Elevation_End'] < elevation]
        blw_water_table.append(ttem_blw)

    ttem_above = pd.concat(abv_water_table)
    ttem_below = pd.concat(blw_water_table)
    return ttem_above, ttem_below
def plot_bootstrap_result(dataframe):
    """
    plot bootstrap result

    :param dataframe:
    :return: plotly fig
    """
    fig_hist = go.Figure()
    fig_hist.data = []
    fig_hist.add_trace(go.Histogram(x=dataframe.fine, name='Fine', marker_color='Blue', opacity=0.75))
    fig_hist.add_trace(go.Histogram(x=dataframe.coarse, name='Coarse', marker_color='Red', opacity=0.75))
    if dataframe.mix.sum() == 0:
        print("skip plot mix because there is no data")
    else:
        fig_hist.add_trace(go.Histogram(x=dataframe.mix, name='Mix', marker_color='Green', opacity=0.75))
    fig_hist.update_layout(paper_bgcolor='white',plot_bgcolor='white',font_color='black')

    return fig_hist
def data_fence(ttem_df,xmin,ymin,xmax,ymax):
    if xmin>xmax:
        raise ValueError('xmin:{} is greater than xmax:{}'.format(xmin,xmax))
    if ymin>ymax:
        raise ValueError('ymin:{} is greater than ymax:{}'.format(xmin,xmax))
    new_ttem_df = ttem_df[(ttem_df['UTMX']>xmin)&(ttem_df['UTMX']<xmax)&(ttem_df['UTMY']>ymin)&(ttem_df['UTMY']<ymax)]
    return new_ttem_df
def filter_line(ttem_df, line_filter):
        ttem_preprocess_line = ttem_df[ttem_df['Line_No'] == int(line_filter)]
        if ttem_preprocess_line.empty:
            raise ValueError('Did not found any data under line_no {}, the line number suppose to be integer'.format(line_filter))
        return ttem_preprocess_line
def resistivity_avg(ttem_df):
    new_1d_df = []
    coordinate_group = ttem_df.groupby(['UTMX','UTMY'])
    for name, group in coordinate_group:
        mean_resistivity = group['Resistivity'].mean()
        output_df = {'UTMX':name[0],'UTMY':name[1],'Mean_Resistivity':mean_resistivity}
        tmp_df = pd.DataFrame([output_df],index=None)
        new_1d_df.append(tmp_df)
    export_df = pd.concat(new_1d_df)
    return  export_df
def lithology_pct(rock_transform_df):
    """
    # Plan to group by the coordinate and generate a new dataframe with only [UTMX, UTMY]
    """
    new_1d_df = []
    coordinate_group = rock_transform_df.groupby(['UTMX','UTMY'])
    for name, group in coordinate_group:
        total_thickness_for_point = sum(group['Thickness'])
        lithology_group = group.groupby('Identity')
        try:
            fine_grain_thickness = lithology_group.get_group('Fine_grain')['Thickness'].sum()
        except KeyError:
            fine_grain_thickness = 0
        try:
            mixed_grain_thickness = lithology_group.get_group('Mix_grain')['Thickness'].sum()
        except KeyError:
            mixed_grain_thickness = 0
        try:
            coarse_grain_thickness = lithology_group.get_group('Coarse_grain')['Thickness'].sum()
        except KeyError:
            coarse_grain_thickness = 0
        tmp_df = pd.DataFrame([{'UTMX':name[0],
                  'UTMY':name[1],
                  'Fine grained ratio':fine_grain_thickness/float(total_thickness_for_point),
                  'mixed_grain_thickness':mixed_grain_thickness/float(total_thickness_for_point),
                  'coarse_grain_thickness':coarse_grain_thickness/float(total_thickness_for_point)}])

        new_1d_df.append(tmp_df)
    export = pd.concat(new_1d_df)
    return export
def block_plot(ttem_df, line_filter=None, fence=None, plot_ft=True):

    """
    This is the function that plot the given dataframe into gridded plot (for better looking)

    :param ttem_df: a data frame contains tTEM data exported from Aarhus workbench
    :param line_filter: In tTEM dataframe you can filter tTEM data by Line_No (default: None)
    :param fence: receive a UTM coordinate to cut data, ex: [xmin, ymin, xmax, ymax] (default: None)
    :param plot_ft: True: plot all parameter in feet, False: plot in meter (default: True)
    :return:
    """
    if line_filter is not None and fence is not None:
        ttem_preprocess_line = filter_line(ttem_df,line_filter)
        ttem_preprocess_fence = data_fence(ttem_preprocess_line,fence[0],fence[1],fence[2],fence[3])
        ttem_preprocessed = ttem_preprocess_fence.copy()
    elif line_filter is not None:
        ttem_preprocess_line = filter_line(ttem_df,line_filter)
        ttem_preprocessed = ttem_preprocess_line.copy()
    elif fence is not None:
        ttem_preprocess_fence = data_fence(ttem_df,fence[0],fence[1],fence[2],fence[3])
        ttem_preprocessed = ttem_preprocess_fence.copy()
    else:
        ttem_preprocessed = ttem_df
    if plot_ft is True:
        m_to_ft_factor = 3.28
        unit = 'ft'
    else:
        m_to_ft_factor = 1
        unit = 'm'
    ttem_preprocessed['distance'] = np.sqrt(ttem_preprocessed['UTMX']**2 + ttem_preprocessed['UTMY']**2)*m_to_ft_factor
    ttem_preprocessed['Elevation_Cell'] = ttem_preprocessed['Elevation_Cell']*m_to_ft_factor
    ttem_preprocessed['Elevation_End'] = ttem_preprocessed['Elevation_End']*m_to_ft_factor
    distance_min = ttem_preprocessed['distance'].min()
    elevation_max = ttem_preprocessed['Elevation_Cell'].max()
    elevation_end_max = ttem_preprocessed['Elevation_End'].max()
    ttem_preprocessed['distance_for_plot'] = ttem_preprocessed['distance'] - distance_min
    ttem_preprocessed['Elevation_Cell_for_plot'] = ttem_preprocessed['Elevation_Cell'] - elevation_max
    ttem_preprocessed['Elevation_End_for_plot'] = ttem_preprocessed['Elevation_End'] - elevation_end_max
    ttem_for_plot = abs(ttem_preprocessed[['distance_for_plot', 'Resistivity','Elevation_Cell_for_plot','Elevation_End_for_plot']])
    x_distance = ttem_for_plot['distance_for_plot'].max()
    y_distance = ttem_for_plot['Elevation_End_for_plot'].max()
    empty_grid = np.full((int((y_distance+10)*10),int(x_distance)),np.nan)
# Fill in the tTEM data by loop through the grid
    for index, line in ttem_for_plot.iterrows():
        distance_round = int(line['distance_for_plot'])
        elevation_cell_round = int(line['Elevation_Cell_for_plot'])
        elevation_end_round = int(line['Elevation_End_for_plot'])
        empty_grid[elevation_cell_round-3*m_to_ft_factor:elevation_end_round+3*m_to_ft_factor,distance_round:distance_round+6*m_to_ft_factor] = np.log10(line['Resistivity'])
    fig = px.imshow(empty_grid, range_color=(0,3),color_continuous_scale=colorRes)
    fig.update_layout(paper_bgcolor='white',plot_bgcolor='white',font_color='black')
    fig.update_layout(
        yaxis=dict(
        title='Elevation ({})'.format(unit),
        #tickmode='linear',
        #tick0=1774,
        #dtick=100
    ),
        xaxis=dict(
        title='Distance ({})'.format(unit)
    )
    )
    fig.update_layout(
        dict(
        xaxis=dict(
            titlefont=dict(
                family="Arial",
                size=50
            ),
            tickfont=dict(
                family="Arial",
                size=45
            )
        ),
        yaxis=dict(
            titlefont=dict(
                family="Arial",
                size=50
            ),
            tickfont=dict(
                family="Arial",
                size=45
            )
        ),)
    )
    fig.update_coloraxes(
        colorbar=dict(
            ticks='outside',
            title='Resistivity',
            tickvals=[0,1,2,3],
            tickfont=dict(
                size=30
            ),
            ticktext=['1','10','100','1000'],
            tickmode='array',
            len=0.5))
    """fig.update_layout(
        title=dict(
            text='tTEM Line100, Northern Parowan Valley',
        ),
        titlefont=dict(
            size=50
        )
    )"""
    fig.show(renderer='browser')
    return fig, ttem_for_plot

In [ ]:
fig, ttem_for_plot = block_plot(ttem_north_df,100)

In [ ]:
ttem_for_plot['Elevation_End_for_plot'].max()-ttem_for_plot['Elevation_Cell_for_plot'].min()

In [ ]:
fig.update_layout(
        yaxis=dict(
        #title='Elevation ({})'.format(unit),
        tickmode='linear',
        tick0=100,
        dtick=500
    ))

In [4]:
ttemall = tt.main.ProcessTTEM(ttem_path=[ttemname_north, ttemname_center],
                                  welllog=welllog,
                                  DOI_path=DOI)

Applying DOI |                                | 75/7968

No layer was excluded
No line was excluded
No point was excluded


Applying DOI |################################| 7968/7968


In [5]:
ttem_all_data = ttemall.data()

In [12]:
mean_resistivity = resistivity_avg(ttem_all_data)

In [14]:
mean_resistivity.to_csv(workdir.joinpath('mean_resistivity.csv'))


In [ ]:
#I do not see any evidence of shallow aquifer present in the centeral Parowan in tTEM data, the water elevation would likely be potential surface. So I decided to not seperate the ttem data.
ttem_center = tt.main.ProcessTTEM(ttem_path=[ttemname_north, ttemname_center],
                                  welllog=welllog,
                                  DOI_path=DOI)
bootstrap=ttem_center.ttem_well_connect(distance=1000)
resistivity_confidence_interval = tt.bootstrap.packup(bootstrap[1]['fine'],bootstrap[1]['mix'], bootstrap[1]['coarse'] )
rock_transform_df = tt.Rock_trans.rock_transform(bootstrap[0],resistivity_confidence_interval)
ttem_rock_trans_figure = go.Figure(tt.plot.generate_trace(rock_transform_df,'rock'))
ttem_rock_trans_figure.show(renderer='browser')


# North part

In [ ]:
#Slice data
ttem_north = tt.main.ProcessTTEM(ttem_path=[ttemname_north],
                                  welllog=welllog,
                                  DOI_path=DOI)
ttem_north_df = ttem_north.data()
ttem_north_df_line_100 = ttem_north_df[ttem_north_df['Line_No'] == 100]
#ttem_north_df_data_fence = data_fence(ttem_north_df_line_100,349221,4209480,350118.90,4212995)
#ttem_north_df_line_100 = ttem_north_df_data_fence.copy()
#Normolize elevation and distance to let it start from zero
ttem_north_df_line_100['distance'] = np.sqrt(ttem_north_df_line_100['UTMX']**2 + ttem_north_df_line_100['UTMY']**2)
distance_min = ttem_north_df_line_100['distance'].min()
elevation_max = ttem_north_df_line_100['Elevation_Cell'].max()
elevation_end_max = ttem_north_df_line_100['Elevation_End'].max()
ttem_north_df_line_100['distance'] = ttem_north_df_line_100['distance'] - distance_min
ttem_north_df_line_100['Elevation_Cell'] = ttem_north_df_line_100['Elevation_Cell'] - elevation_max
ttem_north_df_line_100['Elevation_End'] = ttem_north_df_line_100['Elevation_End'] - elevation_end_max
ttem_north_df_for_plot = abs(ttem_north_df_line_100[['distance', 'Thickness','Resistivity','Elevation_Cell','Elevation_End']])


In [ ]:
block_plot(ttem_north_df,100)

In [ ]:
#create empty grid based on the length and height of the tTEM data
x_distance = ttem_north_df_for_plot['distance'].max()
y_distance = ttem_north_df_for_plot['Elevation_Cell'].max()
empty_grid = np.full((int((y_distance+10)*10),int(x_distance)),np.nan)
# Fill in the tTEM data by loop through the grid
for index, line in ttem_north_df_for_plot.iterrows():
    distance_round = int(line['distance']*3.28)
    elevation_cell_round = int(line['Elevation_Cell']*10*3.28)
    elevation_end_round = int(line['Elevation_End']*10*3.28)
    empty_grid[elevation_cell_round-6:elevation_end_round+6,distance_round:distance_round+6] = np.log10(line['Resistivity'])

In [ ]:
import importlib; importlib.reload(tt.bootstrap)
_, matched_well= tt.bootstrap.select_closest(ttem_north_df_line_100,welllog,distance=1000)
matched_well['distance'] = np.sqrt(matched_well['UTMX']**2 + matched_well['UTMY']**2)
matched_well['distance_for_plot'] = matched_well['distance'] - distance_min
matched_well['Elevation_for_plot'] = matched_well['Elevation'] - elevation_max
matched_well['Elevation_for_plot'] = abs(matched_well['Elevation_for_plot'])

In [ ]:
well13870=matched_well[matched_well['Bore'] == '13870']

In [ ]:
matched_well

In [ ]:
for index,line in well13870.iterrows():
    distance = int(line['distance_for_plot']*3.28)
    elevation = int(line['Elevation_for_plot']*10*3.28)
    empty_grid[elevation-6:elevation+1+6,distance-15:distance+15] = line['Keyword_n']

In [ ]:
fig = px.imshow(empty_grid, range_color=(0,3),color_continuous_scale=colorRes)
fig.update_layout(paper_bgcolor='white',plot_bgcolor='white',font_color='black')
fig.update_layout(yaxis=dict(title='Elevation (m)',tickmode='array',
                             tickvals=[0,100,200,300,400,500,600],
                             ticktext=['1774','1764','1754','1744','1735','1725','1715']),
                  xaxis=dict(title='Distance (m)'))
fig.update_layout(
    dict(
    xaxis=dict(
        titlefont=dict(
            family="Arial",
            size=50
        ),
        tickfont=dict(
            family="Arial",
            size=45
        )
    ),
    yaxis=dict(
        titlefont=dict(
            family="Arial",
            size=50
        ),
        tickfont=dict(
            family="Arial",
            size=45
        )
    ),)
)
fig.update_coloraxes(
    colorbar=dict(
        ticks='outside',
        title='Resistivity',
        tickvals=[0,1,2,3],
        tickfont=dict(
            size=30
        ),
        ticktext=['1','10','100','1000'],
        tickmode='array',
        len=0.5))
"""fig.update_layout(
    title=dict(
        text='tTEM Line100, Northern Parowan Valley',
    ),
    titlefont=dict(
        size=50
    )
)"""
fig.show(renderer='browser')

In [ ]:
ttem_north_df_for_plot['Elevation_End'].max()-ttem_north_df_for_plot['Elevation_Cell'].min()